In [1]:
import os
import sys
import glob
import logging
import traceback

from tkinter import *
import tkinter.ttk as ttk
import tkinter.messagebox as messagebox
from tkinter.filedialog import askopenfilename

from enrich2.plugins.options import ScorerOptions, Option
from enrich2.plugins.options import ScorerOptionsFiles, OptionsFile
from enrich2.plugins.scoring import BaseScorerPlugin

Label = ttk.Label
Entry = ttk.Entry
Button = ttk.Button
Checkbutton = ttk.Checkbutton
OptionMenu = ttk.OptionMenu

FORMAT = '%(levelname)s %(asctime)-15s %(message)s'
formatter = logging.Formatter(FORMAT)
handler = logging.StreamHandler()
handler.setFormatter(formatter)
logging.getLogger().setLevel(logging.INFO)
logging.getLogger().addHandler(handler)


class OptionFrame(LabelFrame):
    def __init__(self, parent, options: ScorerOptions, **kw):
        super().__init__(parent, **kw)       
        self.parent = parent
        self.row = 0
        self.widgets = []
        self.option_vars = []
        self.labels = []        
        self.parse_options(options)        
        self.columnconfigure(0, weight=1)
        self.columnconfigure(1, weight=3)

    def parse_options(self, options: ScorerOptions) -> None:
        if not len(options):
            label_text = "No options found."
            label = Label(self, text=label_text, justify=LEFT, relief=RIDGE)
            label.grid(sticky=EW, columnspan=2, row=self.row, padx=5, pady=5)
            self.rowconfigure(self.row, weight=1)
            self.row += 1
            
            btn = Button(self, text='Show Parameters', command=self.no_options_message)
            btn.grid(row=self.row, column=1, sticky=SE, padx=5, pady=5)
            self.rowconfigure(self.row, weight=1)
            self.row += 1
            return
            
        for option in options:
            try:
                option.validate(option.default)
            except TypeError:
                warn = "The default value for option {} has type" \
                       " '{}' and does not match the specified expected " \
                       "type '{}'. The program may behave unexpectedly."
                messagebox.showwarning(
                    title="Default option type does not match dtype.",
                    message=warn.format(option.name, 
                                        type(option.default).__name__, 
                                        option.dtype.__name__))
            self.create_widget_from_option(option)
            self.rowconfigure(self.row, weight=1)
            self.row += 1
            
        btn = Button(self, text='Show Parameters', command=self.log_parameters)
        btn.grid(row=self.row, column=1, sticky=SE, padx=5, pady=5)
        self.rowconfigure(self.row, weight=1)
        self.row += 1
        return

    def create_widget_from_option(self, option: Option) -> None:
        if option.choices:
            self.make_choice_menu_widget(option)
        elif option.dtype in (str, 'string', 'char', 'chr'):
            self.make_string_entry_widget(option)
        elif option.dtype in ('integer', 'int', int):
            self.make_int_entry_widget(option)
        elif option.dtype in ('float', float):
            self.make_float_entry_widget(option)
        elif option.dtype in ('bool', bool, 'boolean'):
            self.make_bool_entry_widget(option)
        else:
            raise ValueError("Unrecognised attribute in option "
                             "dtype {}.".format(option.dtype))

    def make_choice_menu_widget(self, option: Option) -> None:
        menu_var = StringVar(self)
        menu_var.set(option.default)
        
        label_text = "{}: ".format(option.name)
        label = Label(self, text=label_text, justify=LEFT, relief=RIDGE)
        label.grid(sticky=EW, column=0, row=self.row, padx=5, pady=5)

        choices = option.choices
        if isinstance(choices, list):
            choices = {v:v for v in choices}
        choices = list(choices.keys())
        
        popup_menu = OptionMenu(
            self, menu_var, option.default, *option.choices)
        popup_menu.grid(sticky=E, column=1, row=self.row, padx=5, pady=5)

        self.option_vars.append((option, menu_var))
        self.widgets.append(popup_menu)
        self.labels.append(label)

    def make_entry(self, variable: Variable, option: Option) -> Entry:
        label_text = "{}: ".format(option.name)
        label = Label(self, text=label_text, justify=LEFT, relief=RIDGE)
        label.grid(sticky=EW, column=0, row=self.row, padx=5, pady=5)
        
        bad_input_msg = "Invalid type for entry {}. " \
                        "Expected type {}.".format(
                            option.name, option.dtype.__name__)
        def validate_entry():
            try:
                value = variable.get()
                option.validate(value)
                variable.set(option.dtype(value))
            except (TclError, TypeError):
                messagebox.showwarning(
                    title="Invalid {} Entry".format(option.name),
                    message=bad_input_msg)
                variable.set(option.dtype(option.default))
                return False
            return True

        entry = Entry(
            self, textvariable=variable,
            validate="focusout", validatecommand=validate_entry
        )
        entry.grid(sticky=EW, column=1, row=self.row, padx=5, pady=5)
        self.option_vars.append((option, variable))
        self.widgets.append(entry)
        self.labels.append(label)
        return entry

    def make_string_entry_widget(self, option: Option) -> None:
        variable = StringVar(self)
        variable.set(option.dtype(option.default))
        self.make_entry(variable, option)

    def make_int_entry_widget(self, option: Option) -> None:
        variable = IntVar(self)
        variable.set(option.dtype(option.default))
        self.make_entry(variable, option)

    def make_float_entry_widget(self, option: Option) -> None:
        variable = DoubleVar(self)
        variable.set(option.dtype(option.default))
        self.make_entry(variable, option)

    def make_bool_entry_widget(self, option: Option) -> None:
        variable = BooleanVar(self)
        variable.set(option.default)

        label_text = "{}: ".format(option.name)
        label = Label(self, text=label_text, justify=LEFT, relief=RIDGE)
        label.grid(sticky=EW, column=0, row=self.row, padx=5, pady=5)

        checkbox = Checkbutton(self, variable=variable)
        checkbox.grid(sticky=E, column=1, row=self.row, padx=5, pady=5)

        self.option_vars.append((option, variable))
        self.widgets.append(checkbox)
        self.labels.append(label)

    def min_frame_width(self) -> int:
        col0_max_width = max(*[l.winfo_width() for l in self.labels])
        col1_max_width = max(*[w.winfo_width() for w in self.widgets])
        return col0_max_width + col1_max_width

    def max_frame_width(self) -> int:
        col0_max_width = max(*[l.winfo_width() for l in self.labels])
        col1_max_width = max(*[w.winfo_width() for w in self.widgets])
        return (col0_max_width + col1_max_width) * 2

    def min_frame_height(self) -> int:
        col0_height = sum([l.winfo_height() for l in self.labels])
        col1_height = sum([w.winfo_height() for w in self.widgets])
        return max(*[col0_height, col1_height])

    def max_frame_height(self) -> int:
        col0_height = sum([l.winfo_height() for l in self.labels])
        col1_height = sum([w.winfo_height() for w in self.widgets])
        return max(*[col0_height, col1_height]) * 2

    def get_option_cfg(self) -> dict:
        cfg = {}
        for option, var in self.option_vars:
            value = var.get()
            option.validate(value)
            cfg[option.varname] = var.get()
        return cfg
    
    def no_options_message(self):
        messagebox.showinfo(
                "Nothing to see here...", "No options were found in the loaded plugin script.")
        return                          
    
    def log_parameters(self):
        if not self.option_vars:
            messagebox.showinfo("Nothing to see here...", "Please select files to parse first.")
            return
        
        msg = "Parsing parameters...\n"
        for opt, var in self.option_vars:
            v = var.get()
            if opt.choices:
                v = opt.choices[v]
            n = opt.varname
            n, v, t = str(n), str(v), type(v).__name__
            msg += "{}: (value, type) -> ({}, {})\n".format(n, v, t)
        
        logging.info(msg)
        messagebox.showinfo("Parameters logged!", "See log for loaded parameters.")
    
    def has_options(self):
        return bool(self.labels)

    
options = ScorerOptions()
options.add_option(
    name="Normalization Method",
    varname="logr_method",
    dtype=str,
    default='Wild Type',
    choices={'Wild Type': 'wt', 'Full': 'full', 'Complete': 'complete'},
    tooltip=""
)
options.add_option(
    name="Wild-type string",
    varname="wt_string",
    dtype=str,
    default='Hello World',
    choices=[],
    tooltip=""
)
options.add_option(
    name="Alpha",
    varname="alpha",
    dtype=int,
    default=0,
    choices=[],
    tooltip=""
)
options.add_option(
    name="Beta",
    varname="beta",
    dtype=float,
    default=0.0,
    choices=[],
    tooltip=""
)
options.add_option(
    name="Weighted",
    varname="weighted",
    dtype=bool,
    default=False,
    choices=[],
    tooltip=""
)


root = Tk()
mainframe = OptionFrame(root, options, text='Scorer Options', relief=GROOVE, borderwidth=5, padx=5, pady=5)
mainframe.pack(side=LEFT, expand=YES, fill=BOTH, padx=2, pady=2)
root.mainloop()

In [ ]:
class OptionsFileFrame(LabelFrame):
    def __init__(self, parent, options_files: ScorerOptionsFiles, **config):
        super().__init__(parent, **config)
        self.row = 0
        self.widgets = []
        self.labels = []
        self.scorer_parameter_dicts = []       
        self.make_widgets(options_files)    
        self.columnconfigure(0, weight=3)
        self.columnconfigure(1, weight=1)

    def make_widgets(self, options_files: ScorerOptionsFiles):
        if not len(options_files):
            label_text = "No options found."
            label = Label(self, text=label_text, justify=LEFT, relief=RIDGE)
            label.grid(sticky=EW, columnspan=2, row=self.row, padx=5, pady=5)
            self.rowconfigure(self.row, weight=1)
            self.row += 1
            
            btn = Button(self, text='Show Parameters', command=self.no_options_message)
            btn.grid(row=self.row, column=1, sticky=SE, padx=5, pady=5)
            self.rowconfigure(self.row, weight=1)
            self.row += 1
            return
        
        for options_file in options_files:
            self._make_label(options_file)
            self._make_button(options_file)
            self.rowconfigure(self.row, weight=1)
            self.row += 1
            
        btn = Button(self, text='Show Parameters', command=self.log_parameters)
        btn.grid(row=self.row, column=1, sticky=SE, padx=5, pady=5)
        self.rowconfigure(self.row, weight=1)
        self.row += 1
        return

    def _make_label(self, options_file: OptionsFile):
        label_text = "{}: ".format(options_file.name)
        label = Label(self, text=label_text, justify=LEFT, relief=RIDGE)
        label.grid(row=self.row, column=0, sticky=EW, padx=5, pady=5)

    def _make_button(self, options_file: OptionsFile):
        command = lambda opt=options_file: self.load_file(opt)
        button = Button(self, text='Choose...', command=command)
        button.grid(row=self.row, column=1, sticky=E, padx=5, pady=5)

    def load_file(self, options_file: OptionsFile):
        file_path = askopenfilename()
        if not file_path:
            return
        cfg_error_msg = "There was an error parsing file {}. " \
                        "\n\nPlease see log for details.".format(file_path)
        validation_error_msg = "There was an error during validation. " \
                               "\n\nPlease see log for details."
        type_error = "Parsing functions must return a python dictionary."
        empty_error = "Parsing function returned an empty dictionary"
        success = 'Options successfully parsed and validated from file: \n\n{}'.format(file_path)

        try:
            cfg = options_file.parse_to_dict(file_path)
            if not isinstance(cfg, dict):
                raise TypeError(type_error)
            if not cfg:
                raise ValueError(empty_error)
        except BaseException as error:
            logging.exception(error)
            messagebox.showerror('Parse Error', cfg_error_msg)
            return

        try:
            options_file.validate_cfg(cfg)
            self.scorer_parameter_dicts.append((options_file.name, cfg))
        except BaseException as error:
            logging.exception(error)
            messagebox.showerror('Validation Error', validation_error_msg)
            return
        messagebox.showinfo('Success!', success)

    def get_scorer_parameters_dicts(self):
        return self.scorer_parameter_dicts
                                
    def no_options_message(self):
        msg = "No options were found in the loaded plugin script."
        messagebox.showinfo("Nothing to see here...", msg)
        return
    
    def str_nested(self, data, tab_level=1):
        msg = ""       
        if isinstance(data, list) or isinstance(data, tuple):
            if not data:
                msg += 'Empty Iterable'
            else:
                msg += "-> Iterable"
                for i, item in enumerate(data):
                    msg += '\n' + '\t'*tab_level + '@index {}: '.format(i)
                    msg += self.str_nested(item, tab_level)
                msg += '\n' + '\t'*tab_level + '@end of list'
        elif isinstance(data, dict):
            if not data:
                msg += 'Empty Dictionary'
            else:
                msg += "-> Dictionary"
                for key, value in data.items():
                    msg += '\n' + "\t" * tab_level + "{}: ".format(key)
                    msg += self.str_nested(value, tab_level + 1)    
        else:
            if isinstance(data, str):
                data = "'{}'".format(data)
            dtype = type(data).__name__
            msg += "({}, {})".format(data, dtype)
        return msg
    
    def log_parameters(self):
        if not self.scorer_parameter_dicts:
            messagebox.showinfo(
                "Nothing to see here...", "Please select files to parse first.")
            return
        for name, cfg in self.scorer_parameter_dicts:
            msg = "Parsing parameters...\n{}: (value, type) ".format(name)
            msg += self.str_nested(cfg, tab_level=0)
            logging.info(msg)
        messagebox.showinfo("Parameters logged!", "See log for loaded parameters.")
            
    def has_options(self):
        return bool(self.labels)
        
options = ScorerOptionsFiles()
options.append(OptionsFile.default_json_options_file())
options.append(OptionsFile.default_yaml_options_file())

root = Tk()
mainframe = OptionsFileFrame(root, options, text='Scorer Options', relief=GROOVE, borderwidth=5, padx=5, pady=5)
mainframe.pack(side=LEFT, expand=YES, fill=BOTH, padx=2, pady=2)
root.mainloop() 

In [3]:
class ModuleLoader(object):
    def __init__(self, path):
        if not isinstance(path, str):
            raise TypeError("Argument `path` needs to be a string.")
        if not os.path.exists(path):
            raise IOError("Invalid plugin path {}.".format(path))

        path_to_module_folder = '/'.join(path.split('/')[:-1])
        module_folder = path_to_module_folder.split('/')[-1]
        module_name, ext = os.path.splitext(path.split('/')[-1])
        if ext != '.py':
            raise IOError("Plugin must be a python file.")

        try:
            sys.path.append(path_to_module_folder)
            top_module = __import__("{}.{}".format(module_folder, module_name))
            self.module_name = module_name
            self.module_folder = module_folder
            self.module = getattr(top_module, self.module_name)
        except Exception as err:
            raise ImportError(err)

    def get_module_attrs(self):
        return self.module.__dict__.items()

    def get_attr_from_module(self, name):
        if not hasattr(self.module, name):
            raise AttributeError("Module {} does not have attribute "
                                 "{}.".format(self.module_name, name))
        return getattr(self.module, name)


def load_scoring_class_and_options(path):
    loader = ModuleLoader(path)
    scorers = []
    for attr_name, attr in loader.get_module_attrs():
        if implements_methods(attr) and attr != BaseScorerPlugin:
            scorers.append(attr)
    if len(scorers) < 1:
        raise ImportError("Could not find any classes implementing "
                          "the required BaseScorerPlugin interface.")
    if len(scorers) > 1:
        raise ImportError("Found Multiple classes implementing "
                          "the required BaseScorerPlugin interface.")
   
    scorer_options = ScorerOptions()
    scorer_options_files = ScorerOptionsFiles()
    for attr_name, attr in loader.get_module_attrs():
        if isinstance(attr, ScorerOptions):
            scorer_options = attr
        if isinstance(attr, ScorerOptionsFiles):
            scorer_options_files = attr

    scorer_class = scorers[-1]
    return scorer_class, scorer_options, scorer_options_files


def implements_methods(class_):
    if not hasattr(class_, "_base_name"):
        return False
    if not getattr(class_, "_base_name") == 'BaseScorerPlugin':
        return False
    if not hasattr(class_, "compute_scores"):
        return False
    if not hasattr(class_, "row_apply_function"):
        return False
    return True

In [4]:
class ScorerScriptsDropDown(Frame):

    def __init__(self, parent=None, scripts_dir='plugins/', **config):
        super().__init__(parent, **config)
        self.parent = parent
        self.row = 0
        self.current = ''
        
        self.plugins = {}
        for path in glob.glob("{}/*.py".format(scripts_dir)):
            path = path.replace("\\", '/')
            if "__init__.py" in path:
                continue
            try:    
                result = self.load_plugin(path)
            except Exception as e:
                logging.error(e)
                messagebox.showerror(
                    "Error loading plugin...",
                    "There was an error loading the script:\n\n{}." \
                    "\n\nSee log for details.".format(os.path.join(os.getcwd(), path))
                )
                continue
            klass, options_frame, options_file_frame = result
            self.plugins[klass.__name__] = (
                klass, options_frame, options_file_frame)
        
        if not self.plugins:
            raise ImportError("No plugins could be loaded.")
        
        self.make_drop_down_menu(self.plugins)        
        self.columnconfigure(0, weight=1)
        self.columnconfigure(1, weight=3)
        
    def increment_row(self):
        self.rowconfigure(self.row, weight=1)
        self.row += 1
        
    def load_plugin(self, path):
        result = load_scoring_class_and_options(path)        
        klass, options, options_files = result               
        options_frame = OptionFrame(
            self, options, text='Options', 
            relief=GROOVE, borderwidth=3, padx=5, pady=5
        )    
        options_file_frame = OptionsFileFrame(
            self, options_files, text='Option Files',
            relief=GROOVE, borderwidth=3, padx=5, pady=5
        )        
        return klass, options_frame, options_file_frame
                
    def make_drop_down_menu(self, plugins):
        switch = lambda v: self.update_options_view(v)
        choices = [n for n, _ in plugins.items()]
        menu_var = StringVar(self)
        
        frame = LabelFrame(self, text='Plugins',
            relief=GROOVE, borderwidth=3, padx=5, pady=5
        )
        label = Label(frame, text="Variant Scorer: ", justify=LEFT, relief=RIDGE)
        label.grid(sticky=EW, column=0, row=0, padx=5, pady=5)
        popup_menu = OptionMenu(frame, menu_var, choices[0], *choices, command=switch)
        popup_menu.grid(sticky=E, column=1, row=0, padx=5, pady=5)
        
        frame.grid(sticky=NSEW, columnspan=2, row=self.row, padx=5, pady=5)
        frame.rowconfigure(0, weight=1)
        frame.columnconfigure(0, weight=1)
        frame.columnconfigure(1, weight=3)
        self.row += 1
            
    def update_options_view(self, new_name):
        if self.current:
            _, options_frame, options_file_frame = self.plugins[self.current]
            self.hide_frame(options_frame)
            self.hide_frame(options_file_frame)
        
        _, options_frame, options_file_frame = self.plugins[new_name]
        self.show_frame(options_frame)
        self.show_frame(options_file_frame)
        
        self.current = new_name
        
    def hide_frame(self, frame):
        frame.grid_forget()
        self.row -= 1
        
    def show_frame(self, frame):
        frame.grid(sticky=NSEW, row=self.row, columnspan=2, padx=5, pady=5)
        self.increment_row()

        
root = Tk()
mainframe = ScorerScriptsDropDown(root)
mainframe.pack(side=LEFT, expand=YES, fill=BOTH)
root.mainloop() 

INFO 2017-05-02 18:18:37,493 Parsing parameters...
logr_method: (value, type) -> (wt, str)
weighted: (value, type) -> (True, bool)
ref_seq: (value, type) -> (Input Reference Sequence..., str)
alpha: (value, type) -> (0, int)
beta: (value, type) -> (0.0, float)
threading: (value, type) -> (False, bool)

